# Demostration of ICAROS

## The purpose of this NB is: 

- Selecting events.

- Checking run conditions are normal.

- Computing XY correction maps.


## It is divided into three main sections:

1. __Section A:__ Map building with the automatic script that will be run by the shifter in Canfranc.

2. __Section B:__ Dissection of the script, function by function, to show the performance of each one.

2. __Section C:__ In case the map is produced, map checking.

### Three different optional runs are considered:

- __7546__ (LB phys.)__:__ Regular low background physiscs. It should pass all checks and produce the map.

- __7376__ (HE calib.)__:__ This run was taken during the S1 issue, so the temporal distribution of events is not flat and exception should be raised in the first rate checking.

- __7552__ (LB phys.)__:__  In this case, the gate voltage was changed from 7.7kV to 7.0kV during the run. The gain will take two different values, so there will be two different bands in the plot: E vs Z, and the band selection efficiency should be out of range. 

So, only in first case correction maps will be produced.


In [ ]:
import os
import logging
import warnings
warnings.filterwarnings("ignore")
logging.disable(logging.DEBUG)
this_script_logger = logging.getLogger(__name__)
this_script_logger.setLevel(logging.INFO)

*Add mmkekic remote and checkout clean_krcal branch:*

In [ ]:
%%capture
! cd $HOME/ICAROS; git remote add mmkekic https://github.com/mmkekic/ICAROS.git; git fetch mmkekic; git checkout mmkekic/clean_krcalib; cd -;

_Possible run numbers are:_

In [ ]:
possible_runs = [7546, 7552, 7376]

**Set run number**

In [ ]:
run_number = 7552
assert run_number in possible_runs

*Download data*

In [ ]:
%%capture
! wget https://www.dropbox.com/s/3ky8js2yekh4sqw/kr_emap_xy_100_100_r_6573_time.h5
! wget https://www.dropbox.com/s/5n9gj9wjcz70na7/z_dst_LB_mean_ref.h5
! wget https://www.dropbox.com/s/lsb0xxu6qfqur2g/z_dst_HE_mean_ref.h5

In [ ]:
%%capture

folder_dst       = './'

if run_number==7546:
    dst_file       = 'kdst_{0}_v1.1.0_20190801_krbg_0-1500.h5'.format(run_number)
    config_file    = '$ICARO/krcal/map_builder/config_LBphys.conf'
    ref_histo_file = 'z_dst_LB_mean_ref.h5'
    if not os.path.exists(folder_dst+dst_file):
        ! wget https://www.dropbox.com/s/i2zt3m58tacyqi9/kdst_7546_v1.1.0_20190801_krbg_0-1500.h5
    
elif run_number==7552:
    dst_file       = 'kdst_{0}_v1.1.0_20190801_krbg_1300-2582.h5'.format(run_number)
    config_file    = '$ICARO/krcal/map_builder/config_LBphys.conf'
    ref_histo_file = 'z_dst_LB_mean_ref.h5'
    if not os.path.exists(folder_dst+dst_file):
        ! wget https://www.dropbox.com/s/lx4c62vxxgoope8/kdst_7552_v1.1.0_20190801_krbg_1300-2582.h5

elif run_number==7376:
    dst_file       = 'kdst_{0}_v0.9.9_20190111_krbg_0-3000.h5'.format(run_number)
    config_file    = '$ICARO/krcal/map_builder/config_HEcal.conf'
    ref_histo_file = 'z_dst_HE_mean_ref.h5'
    if not os.path.exists(folder_dst+dst_file):
        ! wget https://www.dropbox.com/s/m1hvjy45b803xe1/kdst_7376_v0.9.9_20190111_krbg_0-3000.h5

*Set input/output variables and configuration file*

In [ ]:
from invisible_cities.core.configure         import configure

output_maps_file = './'

file_bootstrap   = 'kr_emap_xy_100_100_r_6573_time.h5'
map_file_out     = os.path.join(output_maps_file, 'map_{0}.h5'.format(run_number)   )
histo_file_out   = os.path.join(output_maps_file, 'histos_{0}.h5'.format(run_number))

print('Input dst: ', folder_dst + dst_file)
print('Output map file: ', map_file_out)
print('Output histograms file: ', histo_file_out)

ref_Z_histogram = dict(
                    ref_histo_file = ref_histo_file,
                    key_Z_histo    = 'histo_Z_dst') 

config = configure(f'maps {config_file}'.split())

config.update(dict(folder             = folder_dst     ,
                   file_in            = dst_file       ,
                   file_out_map       = map_file_out   ,
                   file_out_hists     = histo_file_out ,
                   ref_Z_histogram    = ref_Z_histogram,
                   run_number         = run_number     ,
                   file_bootstrap_map = file_bootstrap))

****

# Section A: Automatic Script

In [ ]:
from krcal.map_builder.map_builder_functions import map_builder

map_builder(config.as_namespace)

### To check produced map go directly to [Section C.](#SectionC)

***

# Section B: Function by function

#### Imports

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

from krcal.map_builder.map_builder_functions  import load_data
from krcal.map_builder.map_builder_functions  import check_rate_and_hist
from krcal.map_builder.map_builder_functions  import selection_nS_mask_and_checking
from krcal.core.kr_types                      import type_of_signal
from krcal.map_builder.map_builder_functions  import check_Z_dst
from krcal.map_builder.map_builder_functions  import band_selector_and_check
from krcal.map_builder.map_builder_functions  import get_binning_auto
from krcal.map_builder.map_builder_functions  import calculate_map
from krcal.core.kr_types                      import FitType
from krcal.core.selection_functions           import selection_in_band

from krcal.map_builder.map_builder_functions  import check_failed_fits
from krcal.map_builder.map_builder_functions  import regularize_map
from krcal.map_builder.map_builder_functions  import remove_peripheral
from krcal.map_builder.map_builder_functions  import add_krevol
from invisible_cities.reco.corrections_new    import read_maps
from krcal.map_builder.map_builder_functions  import e0_xy_correction

from krcal.core.io_functions                  import write_complete_maps
from krcal.core.selection_functions           import get_time_series_df

from krcal.NB_utils   .xy_maps_functions     import draw_xy_maps
from krcal.core       .map_functions         import relative_errors
from krcal.core       .map_functions         import add_mapinfo

from invisible_cities.reco.corrections_new import apply_all_correction

from krcal.NB_utils.plt_functions                import plot_s1histos
from krcal.NB_utils.plt_functions                import s1d_from_dst
from krcal.NB_utils.plt_functions                import plot_s2histos
from krcal.NB_utils.plt_functions                import s2d_from_dst
from krcal.NB_utils.plt_functions                import plot_selection_in_band

from krcal.core.core_functions                import time_delta_from_time
from krcal.core.fitmap_functions              import time_fcs_df
from krcal.NB_utils.plt_functions             import plot_time_fcs

from invisible_cities.core.configure          import configure

#### Loading data (only events inside detector chamber; i.e. R<=200)

In [ ]:
config_values  = config.as_namespace
quality_ranges = config_values.quality_ranges
print(' Only events with R smaller than {}'.format(quality_ranges['r_max']))
inputs = load_data(input_path         = folder_dst                       ,
                   input_dsts         = dst_file                         ,
                   file_bootstrap_map = file_bootstrap                   ,
                   ref_histo_file     = ref_Z_histogram['ref_histo_file'],
                   key_Z_histo        = ref_Z_histogram['key_Z_histo']   ,
                   quality_ranges     = quality_ranges                   )

dst, bootstrapmap, ref_histos = inputs[0], inputs[1], inputs[2]

### 1st check: rate before selection

In [ ]:
with pd.HDFStore(histo_file_out, "w", complib=str("zlib"), complevel=4) as store_hist:
    check_rate_and_hist(times      = dst.time                ,
                        output_f   = store_hist              ,
                        name_table = "rate_before_sel"       ,
                        n_dev      = config_values.n_dev_rate,
                        bin_size   = 180                     ,
                        normed     = False                   )

In [ ]:
rate_before_hist = pd.read_hdf(histo_file_out, key='rate_before_sel')
plt.figure(figsize=(15,7));
plt.step(rate_before_hist.magnitude, rate_before_hist.entries, where='pre');
plt.xlabel('Time (s)');
plt.ylabel('Entries');
plt.title('Rate before');

## Event selection

### nS1 selection (2nd check)

In [ ]:
nS1_eff_interval = (config_values.nS1_eff_min, config_values.nS1_eff_max)
with pd.HDFStore(histo_file_out, "w", complib=str("zlib"), complevel=4) as store_hist:
    mask_s1 = selection_nS_mask_and_checking(dst        = dst               ,
                                             column     = type_of_signal.nS1,
                                             interval   = nS1_eff_interval  ,
                                             output_f   = store_hist        ,
                                             input_mask = None              ,
                                             nbins_hist = 10                ,
                                             range_hist = (0,10)            ,
                                             norm       = True              )

In [ ]:
s1_hist = pd.read_hdf(histo_file_out, key='nS1')
plt.figure(figsize=(7,5));
plt.step(s1_hist.magnitude, s1_hist.entries, where='pre');
plt.xlabel('nS1');
plt.ylabel('Entries');
plt.title('Number of S1 signals');

In [ ]:
eff = dst[mask_s1].event.nunique()/dst.event.nunique()
print('S1 selection efficiency: ', eff*100, '%')

*Some other S1 plots:*

In [ ]:
s1d = s1d_from_dst(dst)
plot_s1histos(dst,  s1d, bins=20, figsize=(10,10))

### nS2 selection (3rd chech)

In [ ]:
nS2_eff_interval = (config_values.nS2_eff_min, config_values.nS2_eff_max)
with pd.HDFStore(histo_file_out, "r+", complib=str("zlib"), complevel=4) as store_hist:
    mask_s2 = selection_nS_mask_and_checking(dst        = dst               ,
                                             column     = type_of_signal.nS2,
                                             interval   = nS2_eff_interval  ,
                                             output_f   = store_hist        ,
                                             input_mask = mask_s1           ,
                                             nbins_hist = 10                ,
                                             range_hist = (0,10)            ,
                                             norm       = True              )

In [ ]:
s2_hist = pd.read_hdf(histo_file_out, key='nS2')
plt.figure(figsize=(7,5));
plt.step(s2_hist.magnitude, s2_hist.entries, where='pre');
plt.xlabel('nS2');
plt.ylabel('Entries');
plt.title('Number of S2 signals');

In [ ]:
eff = dst[mask_s2].event.nunique()/dst[mask_s1].event.nunique()
print('S2 selection efficiency: ', eff*100, '%')

*Some other S2 plots:*

In [ ]:
s2d = s2d_from_dst(dst)
plot_s2histos(dst, s2d, bins=20, emin=1000, emax=15000, figsize=(10,10))

### 4th check: Z distribution of events is correct

In [ ]:
check_Z_dst(Z_vect   = dst[mask_s2].Z            ,
            ref_hist = ref_histos.Z_dist_hist    ,
            n_sigmas = config_values.nsigmas_Zdst)

In [ ]:
plt.figure(figsize=(8,7));
plt.title('Z distribution');
plt.hist(dst[mask_s2].Z, 10, (0, 550), density=1, histtype='step', alpha=0.7, linewidth=2, label=run_number);
plt.errorbar(ref_histos.Z_dist_hist.bin_centres, ref_histos.Z_dist_hist.bin_entries, yerr=ref_histos.Z_dist_hist.err_bin_entries, fmt='.k', label='reference');
plt.legend();
plt.ylabel('Entries');
plt.xlabel('Z (mm)');

*And XY distribution:*

In [ ]:
plt.figure(figsize=(10,8))
plt.hist2d(dst[mask_s2].X, dst[mask_s2].Y, 100, [(-200,200),(-200,200)])
plt.xlabel('X (mm)');
plt.xlabel('X (mm)');
plt.title('X vs Y');
plt.colorbar();

### Time stamps

In [ ]:
ts, masks = get_time_series_df(20, (dst.time.values[0],dst.time.values[-1]), dst)
fps = time_fcs_df(ts, masks, dst, 
                  nbins_z = 15,
                  nbins_e = 25,
                  range_z = (10, 550),
                  range_e = (7000, 18000),
                  energy  = 'S2e',
                  fit     = FitType.profile)
plot_time_fcs(fps,
              range_chi2  = (0,10),
              range_e0    = (100,20000),
              range_lt    = (1000, 12000),
              figsize     = (12,6))

### Band selection (5th check)

*Geom. correction with bootstrap map is applied before*:

In [ ]:
draw_xy_maps(bootstrapmap)

*Applying geometrical correction:*

In [ ]:
geom_corr = e0_xy_correction(bootstrapmap)
corr = geom_corr(dst[mask_s2].X, dst[mask_s2].Y)

In [ ]:
fig = plt.figure(figsize=(14,8))
plt.subplot(3, 1, 1)
plt.hist2d(dst[mask_s2].Z, dst[mask_s2].S2e, 50, range=[[0,600],[0, 20000]]);

plt.title('S2e uncorrected')
plt.subplot(3, 1, 2)
plt.hist2d(dst[mask_s2].Z, dst[mask_s2].S2e*corr, 50, range=[[0,600],[0, 20000]])
plt.title('S2e corrected boot map')

plt.subplot(3, 1, 3)
plt.hist2d(dst[mask_s2].Z, dst[mask_s2].S2e*corr, 50, range=[[0,600],[9400, 13500]])
plt.title('S2e corrected boot map zoom')
plt.tight_layout()

*Selection:*

In [ ]:
band_sel_params = config_values.band_sel_params
mask_band = band_selector_and_check(dst        = dst         ,
                                    boot_map   = bootstrapmap,
                                    input_mask = mask_s2     ,
                                    **band_sel_params        )

In [ ]:
eff = dst[mask_band].event.nunique()/dst[mask_s2].event.nunique()
print('Band selection efficiency: ', eff*100, '%')

In [ ]:
emaps = e0_xy_correction(bootstrapmap)
E0 = dst[mask_s2].S2e.values * emaps(dst[mask_s2].X.values, dst[mask_s2].Y.values)

sel_krband = np.zeros_like(mask_s2)
sel_krband[mask_s2], fpl, fph, hp, pp = selection_in_band(dst[mask_s2].Z,
                                                           E0,
                                                           range_z = band_sel_params['range_Z'],
                                                           range_e = band_sel_params['range_E'],
                                                           nbins_z = band_sel_params['nbins_z'],
                                                           nbins_e = band_sel_params['nbins_e'],
                                                           nsigma  = band_sel_params['nsigma_sel'])

In [ ]:
plot_selection_in_band(fpl, fph, hp, pp, nsigma = band_sel_params['nsigma_sel'])

*Krypton peak after geometrical correction:*

In [ ]:
corr_band = geom_corr(dst[mask_band].X, dst[mask_band].Y)
fig = plt.figure(figsize=(13,6))
plt.subplot(1, 2, 1)
plt.hist(dst[mask_s2].S2e*corr,  bins = 50, range =(5000,18000))
plt.title('Pre-filter');
plt.xlabel('E (pes)');
plt.ylabel('Entries');
plt.subplot(1, 2, 2)
plt.hist(dst[mask_band].S2e*corr_band,  bins = 50, range =(5000,18000))
plt.title('Post-filter');
plt.xlabel('E (pes)');
plt.ylabel('Entries');

### 6th check: rate after selection

In [ ]:
with pd.HDFStore(histo_file_out, "r+", complib=str("zlib"), complevel=4) as store_hist:
    check_rate_and_hist(times      = dst[mask_band].time     ,
                        output_f   = store_hist              ,
                        name_table = "rate_after_sel"        ,
                        n_dev      = config_values.n_dev_rate,
                        bin_size   = 180                     ,
                        normed     = False                   )

In [ ]:
rate_after_hist = pd.read_hdf(histo_file_out, key='rate_after_sel')
plt.figure(figsize=(15,7));
plt.step(rate_after_hist.magnitude, rate_after_hist.entries, where='pre');
plt.xlabel('Time (s)');
plt.ylabel('Entries');
plt.title('Rate after');

*Temporal distribution of events before and after selection:*

In [ ]:
plt.figure(figsize=(15,7));
plt.fill_between(rate_before_hist.magnitude, rate_before_hist.entries, label='Before', color='orange', alpha=0.5);
plt.fill_between(rate_after_hist.magnitude, rate_after_hist.entries, label='After', color='green', alpha=0.5);
plt.legend();
plt.xlabel('Time (s)');
plt.ylabel('Entries');
plt.title('Rate');
plt.ylim(0, 4000);

In [ ]:
sel_dst = dst[mask_band]

In [ ]:
plt.figure(figsize=(16,6));
plt.subplot(1,2,1);
plt.hist(sel_dst.Z, bins = 50, range =(0,550))
plt.title('Z')
plt.xlabel('Z (mm)');
plt.subplot(1,2,2);
plt.hist2d(sel_dst.X, sel_dst.Y, 100, [(-200,200),(-200,200)])
plt.xlabel('X (mm)');
plt.xlabel('X (mm)');
plt.title('X vs Y');
plt.colorbar();

If all selection cuts and checks are passed successfully, it is time to create the correction map.

## Map production

#### Selection of number of XY bins

In [ ]:
thr_evts_for_sel_map_bins = config_values.thr_evts_for_sel_map_bins
default_n_bins            = config_values.default_n_bins

- If the number of evts is greater than 1e6 -> 100x100 map
- If the number of evts is lower than 1e6 -> 50x50 map

In [ ]:
number_of_bins = get_binning_auto(nevt_sel                = sel_dst.event.nunique()  ,
                                  thr_events_for_map_bins = thr_evts_for_sel_map_bins,
                                  n_bins                  = default_n_bins           )
print('Number of XY bins: ', number_of_bins, '(', sel_dst.event.nunique(), 'events)')

### Map computation

In [ ]:
map_params = config_values.map_params

In [ ]:
warnings.filterwarnings("ignore")
logging.disable(logging.DEBUG)
this_script_logger = logging.getLogger(__name__)
this_script_logger.setLevel(logging.INFO)

In [ ]:
maps = calculate_map(dst        = sel_dst                 ,
                     XYbins     = (number_of_bins         ,
                                   number_of_bins)        ,
                     nbins_z    = map_params['nbins_z']   ,
                     nbins_e    = map_params['nbins_e']   ,
                     z_range    = map_params['z_range']   ,
                     e_range    = map_params['e_range']   ,
                     chi2_range = map_params['chi2_range'],
                     lt_range   = map_params['lt_range']  ,
                     fit_type   = FitType.unbined         ,
                     nmin       = map_params['nmin']      ,
                     x_range    = map_params['x_range']   ,
                     y_range    = map_params['y_range']   )

### Last check: number of failed fits

In [ ]:
maxFailed = map_params['maxFailed']
r_max     = map_params['r_max']

check_failed_fits(maps      = maps          ,
                  maxFailed = maxFailed     ,
                  nbins     = number_of_bins,
                  rmax      = r_max         ,
                  rfid      = r_max         )

#### chi2 regularization and compute relative error maps, instead of absolute error.

In [ ]:
regularized_maps = regularize_map(maps    = maps                    ,
                                  x2range = map_params['chi2_range'])

regularized_maps = relative_errors(am = regularized_maps)

Now, the outer bins are replaced by nans:

In [ ]:
regularized_maps = remove_peripheral(map   = regularized_maps,
                                     nbins = number_of_bins  ,
                                     rmax  = r_max           ,
                                     rfid  = r_max           )

In [ ]:
draw_xy_maps(regularized_maps,
             e0lims  = (7000, 14000),
             ltlims  = (4500, 12000),
             eulims  = (0.0,  1),
             lulims  = (0, 10),
             figsize=(14,10))

#### Mapinfo table is added

In [ ]:
maps = add_mapinfo(asm        = regularized_maps     ,
                   xr         = map_params['x_range'],
                   yr         = map_params['y_range'],
                   nx         = number_of_bins       ,
                   ny         = number_of_bins       ,
                   run_number = run_number           )
print(maps.mapinfo)

#### Temporal evolution table is added

In [ ]:
r_fid         = map_params['r_fid']
nStimeprofile = map_params['nStimeprofile']
add_krevol(maps          = maps                 ,
           dst           = sel_dst              ,
           r_fid         = r_fid                ,
           nStimeprofile = nStimeprofile        ,
           x_range       = map_params['x_range'],
           y_range       = map_params['y_range'],
           XYbins        = (number_of_bins      ,
                            number_of_bins     ))
temp = maps.t_evol

In [ ]:
plt.figure(figsize=(16, 20));
plt.subplot(3,2,1);
plt.title('e0');
plt.errorbar(temp.ts, temp.e0, temp.e0u, fmt='.', linestyle='-');
plt.subplot(3,2,2);
plt.title('lt');
plt.errorbar(temp.ts, temp['lt'], temp['ltu'], fmt='.', linestyle='-');
plt.subplot(3,2,3);
plt.title('dv');
plt.ylim(0.907, 0.920);
plt.errorbar(temp.ts, temp.dv, temp.dvu, fmt='.', linestyle='-');
plt.subplot(3,2,4);
plt.title('s1e');
plt.errorbar(temp.ts, temp.s1e, temp.s1eu, fmt='.', linestyle='-');
plt.subplot(3,2,5);
plt.title('s2e');
plt.errorbar(temp.ts, temp.s2e, temp.s2eu, fmt='.', linestyle='-');
plt.subplot(3,2,6);
plt.title('Nsipm');
plt.errorbar(temp.ts, temp.Nsipm, temp.Nsipmu, fmt='.', linestyle='-');

### Writing final map

In [ ]:
write_complete_maps(asm      = maps        ,
                    filename = map_file_out)

*****

# Section C: Checking map <a id='SectionC'></a>


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from invisible_cities.reco.corrections_new   import read_maps
from invisible_cities.reco.corrections_new   import apply_all_correction
from krcal.NB_utils  .xy_maps_functions      import draw_xy_maps

from krcal.NB_utils.plt_functions             import h1, h2
from krcal.NB_utils.fit_energy_functions      import fit_energy
from krcal.NB_utils.plt_energy_functions      import plot_fit_energy, print_fit_energy
from krcal.NB_utils.plt_energy_functions      import resolution_r_z, plot_resolution_r_z

from krcal.map_builder.map_builder_functions  import e0_xy_correction
from krcal.map_builder.map_builder_functions  import load_data
from krcal.map_builder.map_builder_functions  import apply_cuts
from krcal.map_builder.map_builder_functions  import type_of_signal

### Opening map

In [ ]:
try:
    final_map = read_maps(map_file_out)
except FileNotFoundError:
    print('Please run Section A or B. If you have already done it, the dst is not good enough to produce a map :(')

In [ ]:
draw_xy_maps(final_map,
             e0lims  = (7000, 14000),
             ltlims  = (4500, 12000),
             eulims  = (0.0,  1),
             lulims  = (0, 10),
             figsize=(14,10))

In [ ]:
final_map.mapinfo

### Opening and selecting dst (only if section B is not run)

In [ ]:
try :
    sel_dst;
except NameError:
    config_values = config.as_namespace
    
    inputs = load_data(input_path         = folder_dst                  ,
                       input_dsts         = dst_file                    ,
                       file_bootstrap_map = file_bootstrap              ,
                       quality_ranges     = config_values.quality_ranges,
                       **config_values.ref_Z_histogram                  )

    dst, bootstrapmap, ref_histos = inputs[0], inputs[1], inputs[2]
    
    with pd.HDFStore(histo_file_out, "r+", complib=str("zlib"), complevel=4) as store_hist:
        sel_dst = apply_cuts(dst              = dst                           ,
                             S1_signal        = type_of_signal.nS1            ,
                             nS1_eff_interval = (config_values.nS1_eff_min    ,
                                                 config_values.nS1_eff_max)   ,
                             store_hist_s1    = store_hist                    ,
                             ns1_histo_params = config_values.ns1_histo_params,
                             S2_signal        = type_of_signal.nS2            ,
                             nS2_eff_interval = (config_values.nS2_eff_min    ,
                                                 config_values.nS2_eff_max)   ,
                             store_hist_s2    = store_hist                    ,
                             ns2_histo_params = config_values.ns2_histo_params,
                             nsigmas_Zdst     = config_values.nsigmas_Zdst    ,
                             ref_Z_histo      = ref_histos.
                                                    Z_dist_hist               ,
                             bootstrapmap     = bootstrapmap                  ,
                             band_sel_params  = config_values.band_sel_params )

### Applying corrections to the selected dst

In [ ]:
sel_dst = sel_dst[sel_dst.R<170]

In [ ]:
geom_corr = e0_xy_correction(final_map)
total_correction = apply_all_correction(final_map, apply_temp=True)

corr_geo = geom_corr(sel_dst.X, sel_dst.Y)
corr_tot = total_correction(sel_dst.X, sel_dst.Y, sel_dst.Z, sel_dst.time)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.subplot(3, 1, 1)
plt.hist2d(sel_dst.Z, sel_dst.S2e, 50, [(0,600),(6000,14000)])
plt.title('Raw energy');
plt.subplot(3, 1, 2)
plt.hist2d(sel_dst.Z, sel_dst.S2e*corr_geo, 50, [(0,600),(6000,14000)])
plt.title('Geom. corrected energy');
plt.subplot(3, 1, 3)
plt.hist2d(sel_dst.Z, sel_dst.S2e*corr_tot, 50, [(0,600),(6000,14000)])
plt.title('Total corrected energy');

In [ ]:
e_range = (8000,16000)
zrange = (10,550)
fig = plt.figure(figsize=(14,8))
plt.subplot(1, 2, 1)

nevt = h2(sel_dst.Z, sel_dst.S2e*corr_tot, 30, 30, zrange, e_range, profile=True)
plt.xlabel('Z (mm)');
plt.ylabel('E (pes)');
plt.title('E vs Z');

ax      = fig.add_subplot(1, 2, 2)
(_)     = h1(sel_dst.S2e*corr_tot,  bins = 100, range =e_range, stats=True, lbl = 'E')
plt.xlabel('E (pes)');
plt.ylabel('Entries');
plt.title('E corr');

In [ ]:
fc = fit_energy(sel_dst.S2e*corr_tot, nbins=100, range=(12000, 14000))
plot_fit_energy(fc)
print_fit_energy(fc)

In [ ]:
Ri = (50, 100,150,170)
Zi = (50, 100,200,300,500)

FC, FCE = resolution_r_z(Ri, Zi, sel_dst.R, sel_dst.Z, sel_dst.S2e*corr_tot,
                    enbins = 50,
                    erange = (12000,14000),
                    ixy = (5,4),
                    fdraw  = True,
                    fprint = False,
                    figsize = (18,10)) 

In [ ]:
plot_resolution_r_z(Ri, Zi, FC, FCE, r_range=(2,6))